In [0]:
# !pip install keras

In [0]:
# !pip install tensorflow

In [0]:
# Importe MLflow, ketas y tensorflow
import mlflow
import mlflow.keras
import keras
import tensorflow as tf
import tensorflow.keras as tk
from keras import models
from keras import layers

In [0]:
# mlflow.set_experiment("mnist-feedforward-rony")

In [0]:
# Usaremos argparse para pasarle argumentos a las funciones de entrenamiento
import argparse

parser = argparse.ArgumentParser(description='Entrenamiento de una red feed-forward para el problema de clasificación con datos MNIST en TensorFlow/Keras')
parser.add_argument('--batch_size', '-b', type=int, default=128)
parser.add_argument('--epochs', '-e', type=int, default=5)
parser.add_argument('--learning_rate', '-l', type=float, default=0.05)
parser.add_argument('--num_hidden_units', '-n', type=int, default=512)
parser.add_argument('--num_hidden_layers', '-N', type=int, default=1)
parser.add_argument('--dropout', '-d', type=float, default=0.25)
parser.add_argument('--momentum', '-m', type=float, default=0.85)

_StoreAction(option_strings=['--momentum', '-m'], dest='momentum', nargs=None, const=None, default=0.85, type=<class 'float'>, choices=None, required=False, help=None, metavar=None)

In [0]:
args = parser.parse_args([])

In [0]:
# Usaremos esta función para definir Descenso de Gradiente Estocástico como optimizador
def get_optimizer():
    """
    :return: Keras optimizer
    """
    optimizer = keras.optimizers.SGD(learning_rate=args.learning_rate,momentum=args.momentum, nesterov=True)
    return optimizer

In [0]:
# Obtenemos el dataset MNIST 
mnist = tk.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# normalizamos los datos para que estén entre 0 y 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Esta función define una corrida del modelo, con entrenamiento y 
# registro en MLflow
def run_mlflow(run_name="MLflow CE MNIST"):
    # Iniciamos una corrida de MLflow
    mlflow.start_run(run_name=run_name)
    run = mlflow.active_run()
    # MLflow asigna un ID al experimento y a la corrida
    experimentID = run.info.experiment_id
    runID = run.info.run_uuid
    # reistro automáticos de las métricas de keras
    mlflow.keras.autolog()
    model = models.Sequential()
    #
    # La primera capa de la red transforma las imágenes de un arreglo 2d (28x28 pixels),
    # en un arreglo 1d de 28 * 28 = 784 pixels.
    model.add(layers.Flatten(input_shape=x_train[0].shape))
    # Agregamos capas ocultas a la red
    # en los argumentos: --num_hidden_layers o -N 
    for n in range(0, args.num_hidden_layers):
        # agregamos una capa densa (completamente conectada) con función de activación relu
        model.add(layers.Dense(args.num_hidden_units, activation=tf.nn.relu))
        # agregamos dropout como método de regularización para aleatoriamente descartar una capa
        # si los gradientes son muy pequeños
        model.add(layers.Dropout(args.dropout))
        # capa final con 10 nodos de salida y activación softmax 
        model.add(layers.Dense(10, activation=tf.nn.softmax))
        # Use Scholastic Gradient Descent (SGD) or Adadelta
        # https://keras.io/optimizers/
        optimizer = get_optimizer()

    # compilamos el modelo y definimos la función de pérdida  
    # otras funciones de pérdida comunes para problemas de clasificación
    # 1. sparse_categorical_crossentropy
    # 2. binary_crossentropy
    model.compile(optimizer=optimizer,
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    # entrenamos el modelo
    print("-" * 100)
    model.fit(x_train, y_train, epochs=args.epochs, batch_size=args.batch_size)
    # evaluamos el modelo
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    mlflow.end_run(status='FINISHED')
    return (experimentID, runID)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [0]:
# 1. Aumentar el número de épocas y unidades ocultas
# Entrena más tiempo y con redes más grandes, para ver si mejora la precisión.
args = parser.parse_args(["--epochs", "15", "--num_hidden_units", "1024"])
(experimentID, runID) = run_mlflow()
print(f"MLflow Run completed with run_id {runID} and experiment_id {experimentID}")
print(tf.__version__)
print("-" * 100)

2025/10/26 01:21:50 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.9.2, but the installed version is 3.11.3. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.
/local_disk0/.ephemeral_nfs/envs/pythonEnv-c8903e90-0976-48bc-9bb7-088f12d16c8f/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
W0000 00:00:1761441711.085199    2610 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.

----------------------------------------------------------------------------------------------------


2025-10-26 01:21:52.056884: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 188160000 exceeds 10% of free system memory.


2025/10/26 01:22:53 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: HTTPSConnectionPool(host='us-west-2.storage.cloud.databricks.com', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/836499afcc0b49b4af7421a2da04063f/artifacts/model_summary.txt?v=1&workspaceId=1414358946835367&wkt=cp&userId=4639089670320162&X-Databricks-TTL=3599996&X-Databricks-Consumer-Network-Zone=internet&sn=oregon-prod&X-Databricks-Key-Version=01f0b201-ed28-12b3-b6c3-735f8e1a777b&X-Databricks-Issued=20251026T012152Z&X-Databricks-Signature=AYc93NqMmMuBaUoYUUwkipIIEKAFHBLaNlb4QJGhjWkNy-8lGA (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc29cf89850>: Failed to establish a new connection: [Errno 111] Connection refused'))


Epoch 1/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9112 - loss: 0.3100
Epoch 2/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9585 - loss: 0.1454
Epoch 3/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9700 - loss: 0.1045
Epoch 4/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9761 - loss: 0.0822
Epoch 5/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9807 - loss: 0.0687
Epoch 6/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9831 - loss: 0.0580
Epoch 7/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9860 - loss: 0.0497
Epoch 8/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9883 - loss: 0.0428
Epoch 9/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9887 - loss: 0.0387
Epoch 10/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9905 - loss: 0.0338
Epoch 11/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9919 - loss: 0.0295
Epoch 12/15
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step

2025/10/26 01:23:42 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpnmqytllh/model, flavor: keras). Fall back to return ['keras==3.11.3']. Set logging level to DEBUG to see the full traceback. 
2025/10/26 01:24:45 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: The following failures occurred while uploading one or more artifacts to dbfs:/databricks/mlflow-tracking/1331695653098030/836499afcc0b49b4af7421a2da04063f/artifacts: {'/local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpnmqytllh/model/requirements.txt': 'MlflowException("HTTPSConnectionPool(host=\'us-west-2.storage.cloud.databricks.com\', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/836499afcc0b49b4af7421a2da04063f/artifacts/model/requirements.txt?v=1&workspaceId=14143589468353

313/313 - 0s - 2ms/step - accuracy: 0.9833 - loss: 0.0527
MLflow Run completed with run_id 836499afcc0b49b4af7421a2da04063f and experiment_id 1331695653098030
2.20.0
----------------------------------------------------------------------------------------------------


In [0]:
# 2. Variar el tamaño de batch y la tasa de aprendizaje
# Usa batches más pequeños y un learning rate más bajo, lo que puede estabilizar el entrenamiento.
args = parser.parse_args(["--batch_size", "64", "--learning_rate", "0.01"])
(experimentID, runID) = run_mlflow()
print(f"MLflow Run completed with run_id {runID} and experiment_id {experimentID}")
print(tf.__version__)
print("-" * 100)

2025/10/26 01:24:46 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.9.2, but the installed version is 3.11.3. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.


----------------------------------------------------------------------------------------------------


2025-10-26 01:24:47.267727: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 188160000 exceeds 10% of free system memory.


2025/10/26 01:25:50 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: HTTPSConnectionPool(host='us-west-2.storage.cloud.databricks.com', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/91bf379386534ae6bc580e41339ad159/artifacts/model_summary.txt?v=1&workspaceId=1414358946835367&wkt=cp&userId=4639089670320162&X-Databricks-TTL=3599996&X-Databricks-Consumer-Network-Zone=internet&sn=oregon-prod&X-Databricks-Key-Version=01f0b201-ed28-12b3-b6c3-735f8e1a777b&X-Databricks-Issued=20251026T012447Z&X-Databricks-Signature=AYc93Nri-ujmulW96jDlo6NqPnV2OTatqDgQE8hfEDIYpy6MXA (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc17af4a150>: Failed to establish a new connection: [Errno 111] Connection refused'))


Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8803 - loss: 0.4310
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9352 - loss: 0.2311
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9484 - loss: 0.1792
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9585 - loss: 0.1485
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9644 - loss: 0.1272


2025/10/26 01:26:11 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpy9wrmbxh/model, flavor: keras). Fall back to return ['keras==3.11.3']. Set logging level to DEBUG to see the full traceback. 
2025/10/26 01:27:15 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: The following failures occurred while uploading one or more artifacts to dbfs:/databricks/mlflow-tracking/1331695653098030/91bf379386534ae6bc580e41339ad159/artifacts: {'/local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpy9wrmbxh/model/requirements.txt': 'MlflowException("HTTPSConnectionPool(host=\'us-west-2.storage.cloud.databricks.com\', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/91bf379386534ae6bc580e41339ad159/artifacts/model/requirements.txt?v=1&workspaceId=14143589468353

313/313 - 0s - 1ms/step - accuracy: 0.9675 - loss: 0.1108
MLflow Run completed with run_id 91bf379386534ae6bc580e41339ad159 and experiment_id 1331695653098030
2.20.0
----------------------------------------------------------------------------------------------------


In [0]:
# 3. Probar con varias capas ocultas y dropout más agresivo
# Profundiza la red con más capas y agrega regularización fuerte para evitar sobreajuste.
args = parser.parse_args(["--num_hidden_layers", "3", "--dropout", "0.5"])
(experimentID, runID) = run_mlflow()
print(f"MLflow Run completed with run_id {runID} and experiment_id {experimentID}")
print(tf.__version__)
print("-" * 100)

2025/10/26 01:27:17 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.9.2, but the installed version is 3.11.3. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.


----------------------------------------------------------------------------------------------------


2025-10-26 01:27:18.010064: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 188160000 exceeds 10% of free system memory.


2025/10/26 01:28:19 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: HTTPSConnectionPool(host='us-west-2.storage.cloud.databricks.com', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/64d52a3c120848a8b7c7212bd4528b34/artifacts/model_summary.txt?v=1&workspaceId=1414358946835367&wkt=cp&userId=4639089670320162&X-Databricks-TTL=3599997&X-Databricks-Consumer-Network-Zone=internet&sn=oregon-prod&X-Databricks-Key-Version=01f0b201-ed28-12b3-b6c3-735f8e1a777b&X-Databricks-Issued=20251026T012718Z&X-Databricks-Signature=AYc93NqgBsNZ6JVs6kBw8VIRlPCrtRGXeFMsJ-pzhRgv_mIYXQ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc17ae9f8c0>: Failed to establish a new connection: [Errno 111] Connection refused'))


Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1099 - loss: 2.3021
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1098 - loss: 2.3021
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.1110 - loss: 2.3018
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1117 - loss: 2.3018
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.1114 - loss: 2.3018


2025/10/26 01:28:40 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpnt12ws60/model, flavor: keras). Fall back to return ['keras==3.11.3']. Set logging level to DEBUG to see the full traceback. 
2025/10/26 01:29:44 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: The following failures occurred while uploading one or more artifacts to dbfs:/databricks/mlflow-tracking/1331695653098030/64d52a3c120848a8b7c7212bd4528b34/artifacts: {'/local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpnt12ws60/model/requirements.txt': 'MlflowException("HTTPSConnectionPool(host=\'us-west-2.storage.cloud.databricks.com\', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/64d52a3c120848a8b7c7212bd4528b34/artifacts/model/requirements.txt?v=1&workspaceId=14143589468353

313/313 - 0s - 2ms/step - accuracy: 0.1135 - loss: 2.3016
MLflow Run completed with run_id 64d52a3c120848a8b7c7212bd4528b34 and experiment_id 1331695653098030
2.20.0
----------------------------------------------------------------------------------------------------


In [0]:
# 4. Ajustar el optimizador con mayor momentum
# Un momentum alto acelera la convergencia en valles amplios de la función de pérdida.
args = parser.parse_args(["--epochs", "10", "--momentum", "0.95", "--learning_rate", "0.02"])
(experimentID, runID) = run_mlflow()
print(f"MLflow Run completed with run_id {runID} and experiment_id {experimentID}")
print(tf.__version__)
print("-" * 100)

2025/10/26 01:29:45 WARNING mlflow.utils.autologging_utils: MLflow keras autologging is known to be compatible with 3.0.2 <= keras <= 3.9.2, but the installed version is 3.11.3. If you encounter errors during autologging, try upgrading / downgrading keras to a compatible version, or try upgrading MLflow.


----------------------------------------------------------------------------------------------------


2025-10-26 01:29:46.316215: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 188160000 exceeds 10% of free system memory.


2025/10/26 01:30:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: HTTPSConnectionPool(host='us-west-2.storage.cloud.databricks.com', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/00a29ae29d034a50a48c10d397ac6a71/artifacts/model_summary.txt?v=1&workspaceId=1414358946835367&wkt=cp&userId=4639089670320162&X-Databricks-TTL=3599997&X-Databricks-Consumer-Network-Zone=internet&sn=oregon-prod&X-Databricks-Key-Version=01f0b201-ed28-12b3-b6c3-735f8e1a777b&X-Databricks-Issued=20251026T012946Z&X-Databricks-Signature=AYc93NqRTVyfCV2t4YQfDWutNljSseguW4F0jGd7oKWMyIfafg (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fc17af78b00>: Failed to establish a new connection: [Errno 111] Connection refused'))


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9013 - loss: 0.3335
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9572 - loss: 0.1477
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9685 - loss: 0.1086
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9753 - loss: 0.0851
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9786 - loss: 0.0711
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9824 - loss: 0.0601
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9847 - loss: 0.0530
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9856 - loss: 0.0479
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9873 - loss: 0.0416
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9893 - loss: 0.0369


2025/10/26 01:31:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpnulzz466/model, flavor: keras). Fall back to return ['keras==3.11.3']. Set logging level to DEBUG to see the full traceback. 
2025/10/26 01:32:15 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during keras autologging: The following failures occurred while uploading one or more artifacts to dbfs:/databricks/mlflow-tracking/1331695653098030/00a29ae29d034a50a48c10d397ac6a71/artifacts: {'/local_disk0/user_tmp_data/spark-c8903e90-0976-48bc-9bb7-08/tmpnulzz466/model/requirements.txt': 'MlflowException("HTTPSConnectionPool(host=\'us-west-2.storage.cloud.databricks.com\', port=443): Max retries exceeded with url: /api/2.0/fs/files/WorkspaceInternal/Jobs/mlflow-tracking/1331695653098030/00a29ae29d034a50a48c10d397ac6a71/artifacts/model/requirements.txt?v=1&workspaceId=14143589468353

313/313 - 0s - 1ms/step - accuracy: 0.9812 - loss: 0.0591
MLflow Run completed with run_id 00a29ae29d034a50a48c10d397ac6a71 and experiment_id 1331695653098030
2.20.0
----------------------------------------------------------------------------------------------------
